In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.plotting.misc import plot_solar_system
from poliastro.threebody.flybys import compute_flyby
from astropy.time import Time, TimeDelta

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [216]:
## Main dates (1 набор дат)
date_launch = Time("2026-06-07", scale="utc").tdb
date_arrival = Time("2027-12-13", scale="utc").tdb

In [217]:
Eros_osc = Orbit.from_sbdb("Eros")  # Eros SPK-ID
Eros_osc
Eros_osc.epoch.iso

'2022-01-21 00:01:09.184'

In [218]:
Eros = Ephem.from_horizons("Eros", time_range(date_launch, end=date_arrival), plane=Planes.EARTH_EQUATOR)
Eros

Ephemerides at 50 epochs from 2026-06-07 00:01:09.185 (TDB) to 2027-12-13 00:01:09.183 (TDB)

In [219]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
earth_1

Ephemerides at 50 epochs from 2026-06-07 00:01:09.185 (TDB) to 2026-06-07 00:01:09.185 (TDB)

In [220]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_eros_1 = Orbit.from_ephem(Sun, Eros, date_arrival)

In [221]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_eros_1)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

In [236]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth")
plotter.plot_ephem(Eros, date_arrival, label="Eros")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=287 * u.deg), color="blue", label="Transfer orbit Earth-Eros"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [223]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)

In [237]:
# Фазовые координаты Земли

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

-37230992.738869734 km
-135843674.5484786 km
-58867657.76067638 km

[28.41635432 -6.75783283 -2.93099165] km / s


In [238]:
# Фазовые координаты Ерос

print (f"{ss_eros_1.r [0].to(u.km)}")
print (f"{ss_eros_1.r [1].to(u.km)}")
print (f"{ss_eros_1.r [2].to(u.km)}"'\n')

print (ss_eros_1.v.to(u.km / u.s))

-133132762.0195103 km
101448906.85946935 km
33587664.607221246 km

[-20.00995019 -18.50150272 -14.12260181] km / s


In [226]:
nh_tof_1 = date_arrival - date_launch # время полета Земля1 - Венера

(v0_Ea1, vk_Eros), = iod.lambert(Sun.k, r_earth, ss_eros_1.r, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=553.9999999841842>

In [227]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 7.69409871 km / s>

In [228]:
V_inf = ((v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity [ 1.95903516, -7.37496213, -0.98553017] km / s>

In [229]:
# Гиперболический избыток скорости при подлете от Эрос

V_inf_end_vec = (norm(vk_Eros - ss_eros_1.v)).to(u.km / u.s)
V_inf_end_vec

<Quantity 11.77863618 km / s>

In [230]:
V_inf_end_vec = (vk_Eros - ss_eros_1.v).to(u.km / u.s)
V_inf_end_vec

<Quantity [ 8.54894976, -7.71125281,  2.48763108] km / s>

In [231]:
scalar_EEr = r_earth[0]*ss_eros_1.r[0] + r_earth[1]*ss_eros_1.r[1] +r_earth[2]*ss_eros_1.r[2] 

In [232]:
fi_EEr  = (scalar_EEr/norm(r_earth)/norm(ss_eros_1.r))

In [233]:
np.degrees(np.arccos(fi_EEr))

<Quantity 114.48588553 deg>

In [239]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 7.6940987 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/R)
dV * u.km / u.s

<Quantity 5.64729428 km / s>

In [241]:
print (ss_earth_1.nu)
print (ss_eros_1.nu)

ist_an_1 = 0.3345935105889941
ist_an_2 = 2.4542079225259217 

exs_trans = (norm(ss_eros_1.r)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(ss_eros_1.r)/norm(r_earth) * cos(ist_an_2)))
print (f"Эксцентриситет {exs_trans}")

2.4542079225259217 rad
0.3345935105889941 rad
Эксцентриситет 0.06539092773795506


In [242]:
p_trans = norm(ss_eros_1.r)*(1+cos(ist_an_1)*exs_trans)
print (f"Фокальный параметр {p_trans.to(u.km)}")

Фокальный параметр 181261423.1951755 km


In [243]:
a_trans = p_trans/(1 - exs_trans*exs_trans)
print (f"Большая полуось {a_trans.to(u.km)}")

Большая полуось 182039820.63147604 km


In [244]:
# Геоцентрический участок траектории 

V_inf_geo = 7.6940987
R = r+200

V0 = sqrt (mu/(R))
e_geo = 1 + (R*V_inf_geo*V_inf_geo)/mu
p_geo = R*(1+e_geo)
a_geo = p_geo/(e_geo*e_geo-1)
n_geo = sqrt (mu/a_geo/a_geo/a_geo)
V_geo = sqrt ((2*mu)/R+V_inf_geo*V_inf_geo) 
dV = V_geo - V0

print (f"Скорость на круговой орбите {V0}")
print (f"Эксцентриситет {e_geo}")
print (f"Фокальный параметр {p_geo}")
print (f"Большая полуось {a_geo}")
print (f"Среднее движение {n_geo}")
print (f"Cкорость КА после импульса разгонного блока {V_geo}")
print (f"Импульс скорости, потребный для формирования гиперболической траектории {dV}")

Скорость на круговой орбите 7.788487982042219
Эксцентриситет 1.975908718921728
Фокальный параметр 19554.696192034673
Большая полуось 6733.211695516189
Среднее движение 0.001142708568798407
Cкорость КА после импульса разгонного блока 13.435782258513044
Импульс скорости, потребный для формирования гиперболической траектории 5.6472942764708245
